In [99]:
import pandas as pd
import numpy as np
from datetime import datetime
import random
import requests
import datetime





In [71]:
# Set seed for reproducibility
random.seed(42)
np.random.seed(42)

In [72]:
data = {
    'datetime': [
        '01-07-2015 05:00', '01-07-2015 06:00', '01-07-2015 07:00', '01-07-2015 08:00',
        '01-07-2015 09:00', '01-07-2015 10:00', '01-07-2015 11:00', '01-07-2015 12:00',
        '01-07-2015 13:00', '01-07-2015 14:00', '01-07-2015 15:00', '01-07-2015 16:00',
        '01-07-2015 17:00', '01-07-2015 18:00', '01-07-2015 19:00', '01-07-2015 20:00',
        '01-07-2015 21:00', '01-07-2015 22:00', '01-07-2015 23:00', '02-07-2015 00:00',
        '02-07-2015 01:00', '02-07-2015 02:00', '02-07-2015 03:00', '02-07-2015 04:00',
        '02-07-2015 05:00', '02-07-2015 06:00', '02-07-2015 07:00', '02-07-2015 08:00',
        '02-07-2015 09:00', '02-07-2015 10:00', '02-07-2015 11:00', '02-07-2015 12:00',
        '02-07-2015 13:00', '02-07-2015 14:00', '02-07-2015 15:00', '02-07-2015 16:00',
        '02-07-2015 17:00', '02-07-2015 18:00', '02-07-2015 19:00', '02-07-2015 20:00',
        '02-07-2015 21:00', '02-07-2015 22:00', '02-07-2015 23:00', '03-07-2015 00:00',
        '03-07-2015 01:00', '03-07-2015 02:00', '03-07-2015 03:00', '03-07-2015 04:00',
        '03-07-2015 05:00', '03-07-2015 06:00', '03-07-2015 07:00', '03-07-2015 08:00',
        '03-07-2015 09:00', '03-07-2015 10:00', '03-07-2015 11:00', '03-07-2015 12:00',
        '03-07-2015 13:00', '03-07-2015 14:00', '03-07-2015 15:00', '03-07-2015 16:00',
        '03-07-2015 17:00', '03-07-2015 18:00', '03-07-2015 19:00', '03-07-2015 20:00',
        '03-07-2015 21:00', '03-07-2015 22:00', '03-07-2015 23:00', '04-07-2015 00:00',
        '04-07-2015 01:00', '04-07-2015 02:00', '04-07-2015 03:00', '04-07-2015 04:00',
        '04-07-2015 05:00', '04-07-2015 06:00', '04-07-2015 07:00', '04-07-2015 08:00',
        '04-07-2015 09:00', '04-07-2015 10:00', '04-07-2015 11:00', '04-07-2015 12:00',
        '04-07-2015 13:00', '04-07-2015 14:00', '04-07-2015 15:00', '04-07-2015 16:00',
        '04-07-2015 17:00', '04-07-2015 18:00', '04-07-2015 19:00', '04-07-2015 20:00',
        '04-07-2015 21:00', '04-07-2015 22:00', '04-07-2015 23:00', '05-07-2015 00:00',
        '05-07-2015 01:00', '05-07-2015 02:00', '05-07-2015 03:00', '05-07-2015 04:00',
        '05-07-2015 05:00', '05-07-2015 06:00', '05-07-2015 07:00', '05-07-2015 08:00',
        '05-07-2015 09:00', '05-07-2015 10:00', '05-07-2015 11:00', '05-07-2015 12:00',
        '05-07-2015 13:00', '05-07-2015 14:00', '05-07-2015 15:00', '05-07-2015 16:00',
        '05-07-2015 17:00', '05-07-2015 18:00', '05-07-2015 19:00', '05-07-2015 20:00',
        '05-07-2015 21:00', '05-07-2015 22:00', '05-07-2015 23:00', '06-07-2015 00:00',
        '06-07-2015 01:00', '06-07-2015 02:00', '06-07-2015 03:00', '06-07-2015 04:00'
    ],
    'megawatt_hours': [
        162827, 335153, 333837, 398386, 388954, 392487, 404647, 422227, 442131, 464371,
        491512, 518043, 540514, 560701, 576400, 587405, 594477, 595242, 586317, 572656,
        554682, 543332, 520871, 485722, 453284, 429199, 407007, 395194, 387654, 390157,
        401643, 417369, 437741, 459562, 485186, 509369, 529809, 546611, 558445, 565833,
        569351, 569616, 561747, 549867, 533110, 522864, 501272, 470322, 440084, 415203,
        394227, 383147, 373941, 371552, 373312, 380160, 396051, 418789, 444329, 466734,
        487268, 504422, 518545, 528781, 536381, 539823, 534988, 523996, 510066, 500554,
        482726, 455366, 428928, 403685, 384331, 373639, 363002, 357789, 355088, 356956,
        371087, 396040, 423956, 450542, 472985, 490561, 502921, 512645, 519700, 521413,
        515522, 501977, 482706, 469315, 454492, 436638, 414372, 391943, 374853, 366221,
        357581, 352403, 348599, 351157, 368197, 393860, 423719, 452969, 480469, 503990,
        521951, 535678, 545454, 550003, 547755, 538399, 524023, 515913, 497970, 467812
    ]
}


In [73]:
data

{'datetime': ['01-07-2015 05:00',
  '01-07-2015 06:00',
  '01-07-2015 07:00',
  '01-07-2015 08:00',
  '01-07-2015 09:00',
  '01-07-2015 10:00',
  '01-07-2015 11:00',
  '01-07-2015 12:00',
  '01-07-2015 13:00',
  '01-07-2015 14:00',
  '01-07-2015 15:00',
  '01-07-2015 16:00',
  '01-07-2015 17:00',
  '01-07-2015 18:00',
  '01-07-2015 19:00',
  '01-07-2015 20:00',
  '01-07-2015 21:00',
  '01-07-2015 22:00',
  '01-07-2015 23:00',
  '02-07-2015 00:00',
  '02-07-2015 01:00',
  '02-07-2015 02:00',
  '02-07-2015 03:00',
  '02-07-2015 04:00',
  '02-07-2015 05:00',
  '02-07-2015 06:00',
  '02-07-2015 07:00',
  '02-07-2015 08:00',
  '02-07-2015 09:00',
  '02-07-2015 10:00',
  '02-07-2015 11:00',
  '02-07-2015 12:00',
  '02-07-2015 13:00',
  '02-07-2015 14:00',
  '02-07-2015 15:00',
  '02-07-2015 16:00',
  '02-07-2015 17:00',
  '02-07-2015 18:00',
  '02-07-2015 19:00',
  '02-07-2015 20:00',
  '02-07-2015 21:00',
  '02-07-2015 22:00',
  '02-07-2015 23:00',
  '03-07-2015 00:00',
  '03-07-2015 01:00'

In [74]:
# Create DataFrame
df = pd.DataFrame(data)

In [75]:
df

,datetime,megawatt_hours
0,01-07-2015 05:00,162827
1,01-07-2015 06:00,335153
2,01-07-2015 07:00,333837
3,01-07-2015 08:00,398386
4,01-07-2015 09:00,388954
...,...,...
115,06-07-2015 00:00,538399
116,06-07-2015 01:00,524023
117,06-07-2015 02:00,515913
118,06-07-2015 03:00,497970


In [76]:
# Convert datetime to proper format
df['datetime'] = pd.to_datetime(df['datetime'], format='%d-%m-%Y %H:%M')

In [77]:
df

,datetime,megawatt_hours
0,2015-07-01 05:00:00,162827
1,2015-07-01 06:00:00,335153
2,2015-07-01 07:00:00,333837
3,2015-07-01 08:00:00,398386
4,2015-07-01 09:00:00,388954
...,...,...
115,2015-07-06 00:00:00,538399
116,2015-07-06 01:00:00,524023
117,2015-07-06 02:00:00,515913
118,2015-07-06 03:00:00,497970


In [78]:
# Sort by datetime
df = df.sort_values('datetime').reset_index(drop=True)

In [79]:
# STEP 2 — Define location (latitude, longitude)
# Example: Odisha, India

latitude = 20.9517
longitude = 85.0985

In [80]:
# STEP 3 — Get weather data using Open-Meteo API

In [81]:
start_date = df['datetime'].min().strftime("%Y-%m-%d")
end_date = df['datetime'].max().strftime("%Y-%m-%d")

url = (
    f"https://archive-api.open-meteo.com/v1/archive?"
    f"latitude={latitude}&longitude={longitude}"
    f"&start_date={start_date}&end_date={end_date}"
    f"&hourly=temperature_2m,relative_humidity_2m"
)


In [82]:
response = requests.get(url)
weather_data = response.json()

In [83]:
weather_data

{'latitude': 20.984182,
 'longitude': 85.07099,
 'generationtime_ms': 2.649068832397461,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 118.0,
 'hourly_units': {'time': 'iso8601',
  'temperature_2m': '°C',
  'relative_humidity_2m': '%'},
 'hourly': {'time': ['2015-07-01T00:00',
   '2015-07-01T01:00',
   '2015-07-01T02:00',
   '2015-07-01T03:00',
   '2015-07-01T04:00',
   '2015-07-01T05:00',
   '2015-07-01T06:00',
   '2015-07-01T07:00',
   '2015-07-01T08:00',
   '2015-07-01T09:00',
   '2015-07-01T10:00',
   '2015-07-01T11:00',
   '2015-07-01T12:00',
   '2015-07-01T13:00',
   '2015-07-01T14:00',
   '2015-07-01T15:00',
   '2015-07-01T16:00',
   '2015-07-01T17:00',
   '2015-07-01T18:00',
   '2015-07-01T19:00',
   '2015-07-01T20:00',
   '2015-07-01T21:00',
   '2015-07-01T22:00',
   '2015-07-01T23:00',
   '2015-07-02T00:00',
   '2015-07-02T01:00',
   '2015-07-02T02:00',
   '2015-07-02T03:00',
   '2015-07-02T04:00',
   '2015-07-02T05:00',
   '2015

In [84]:
weather_df = pd.DataFrame({
    "datetime": pd.to_datetime(weather_data["hourly"]["time"]),
    "temperature": weather_data["hourly"]["temperature_2m"],
    "humidity": weather_data["hourly"]["relative_humidity_2m"]
})


In [85]:
weather_df

,datetime,temperature,humidity
0,2015-07-01 00:00:00,24.8,95
1,2015-07-01 01:00:00,25.1,94
2,2015-07-01 02:00:00,25.8,90
3,2015-07-01 03:00:00,26.2,87
4,2015-07-01 04:00:00,27.4,82
...,...,...,...
139,2015-07-06 19:00:00,26.5,89
140,2015-07-06 20:00:00,26.0,92
141,2015-07-06 21:00:00,25.8,93
142,2015-07-06 22:00:00,25.6,94


In [86]:
df1 = pd.read_csv(r"C:\Users\TPWODL\New folder_Content\TimeSeriesForecasting_End_To_End\data\raw\Energy Demand Forcasting.csv")

In [87]:
df1

,date,megawatthours
0,2015-07-01 05:00:00,162827
1,2015-07-01 06:00:00,335153
2,2015-07-01 07:00:00,333837
3,2015-07-01 08:00:00,398386
4,2015-07-01 09:00:00,388954
...,...,...
58931,2022-03-21 16:00:00,433344
58932,2022-03-21 17:00:00,429156
58933,2022-03-21 18:00:00,426496
58934,2022-03-21 19:00:00,423393


In [88]:
df1.tail()

,date,megawatthours
58931,2022-03-21 16:00:00,433344
58932,2022-03-21 17:00:00,429156
58933,2022-03-21 18:00:00,426496
58934,2022-03-21 19:00:00,423393
58935,2022-03-21 20:00:00,420157


In [89]:
df1.shape


(58936, 2)

In [102]:
df1 = df1.sort_values("date")

In [104]:
df1['date'] = pd.to_datetime(df1['date'])

In [105]:
start_date1 = df1['date'].min().strftime("%Y-%m-%d")
end_date1 = df1['date'].max().strftime("%Y-%m-%d")

url = (
    f"https://archive-api.open-meteo.com/v1/archive?"
    f"latitude={latitude}&longitude={longitude}"
    f"&start_date={start_date1}&end_date={end_date1}"
    f"&hourly=temperature_2m,relative_humidity_2m"
)

In [106]:
start_date1

'2015-07-01'

In [107]:
end_date1

'2022-03-21'

In [108]:
response_01 = requests.get(url)

In [109]:
weather_data_01 = response_01.json()

In [114]:
weather_data_01

{'latitude': 20.984182,
 'longitude': 85.07099,
 'generationtime_ms': 148.96094799041748,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 118.0,
 'hourly_units': {'time': 'iso8601',
  'temperature_2m': '°C',
  'relative_humidity_2m': '%'},
 'hourly': {'time': ['2015-07-01T00:00',
   '2015-07-01T01:00',
   '2015-07-01T02:00',
   '2015-07-01T03:00',
   '2015-07-01T04:00',
   '2015-07-01T05:00',
   '2015-07-01T06:00',
   '2015-07-01T07:00',
   '2015-07-01T08:00',
   '2015-07-01T09:00',
   '2015-07-01T10:00',
   '2015-07-01T11:00',
   '2015-07-01T12:00',
   '2015-07-01T13:00',
   '2015-07-01T14:00',
   '2015-07-01T15:00',
   '2015-07-01T16:00',
   '2015-07-01T17:00',
   '2015-07-01T18:00',
   '2015-07-01T19:00',
   '2015-07-01T20:00',
   '2015-07-01T21:00',
   '2015-07-01T22:00',
   '2015-07-01T23:00',
   '2015-07-02T00:00',
   '2015-07-02T01:00',
   '2015-07-02T02:00',
   '2015-07-02T03:00',
   '2015-07-02T04:00',
   '2015-07-02T05:00',
   '201